# VacationPy
------------------------

### Excercise Dependencies

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from scipy.stats import linregress
import time
import json

# Import API key
from api_keys import g_key

# Excercise data files
csv_path = os.path.join("..", "PyWeather", "output_data/cities.csv")

# Read the csv from Excercise PyWeather
Cities_weather_df = pd.read_csv(csv_path)

#Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Display of DataFrame
Cities_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,port alfred,100,ZA,1584322882,98,-33.59,26.89,62.996,4.81
1,koulamoutou,52,GA,1584322884,97,-1.13,12.47,71.204,0.66
2,tabuk,82,PH,1584322419,74,17.42,121.44,76.514,5.15
3,nikolskoye,75,RU,1584322392,55,59.70,30.79,33.800,4.00
4,fortuna,20,US,1584322458,70,40.60,-124.16,48.200,4.60
...,...,...,...,...,...,...,...,...,...
560,presidencia roque saenz pena,64,AR,1584324185,95,-26.79,-60.44,71.330,2.16
561,borujerd,0,IR,1584324187,89,33.90,48.75,33.746,1.77
562,kenora,90,CA,1584324189,85,49.82,-94.43,24.800,2.60
563,marica,0,BR,1584324042,78,-22.92,-42.82,80.600,3.10


### Humidity Heatmap
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Storing coordinates 
coordinates = Cities_weather_df[["Lat","Lng"]]

# Storing humidity
humidity = Cities_weather_df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(10.0, 10.0), zoom_level=1.9)

# Defining and adding heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=150,
                                 point_radius=3.1)

fig.add_layer(heat_layer)

# Map layout and display

#figure_layout = {'width': '400px','height': '400px','border': '1px solid black',
                 #'padding': '1px','margin': '0 auto 0 auto'}

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
#Defining my ideal weather
my_weather_df = Cities_weather_df.loc[(Cities_weather_df["Max Temp"]>=65) & (Cities_weather_df["Max Temp"]< 75) &
                                      (Cities_weather_df["Wind Speed"] <20) & (Cities_weather_df["Cloudiness"]==0)]

#Dropping null values

my_weather = Cities_weather_df.dropna()

my_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,port alfred,100,ZA,1584322882,98,-33.59,26.89,62.996,4.81
1,koulamoutou,52,GA,1584322884,97,-1.13,12.47,71.204,0.66
2,tabuk,82,PH,1584322419,74,17.42,121.44,76.514,5.15
3,nikolskoye,75,RU,1584322392,55,59.70,30.79,33.800,4.00
4,fortuna,20,US,1584322458,70,40.60,-124.16,48.200,4.60
...,...,...,...,...,...,...,...,...,...
560,presidencia roque saenz pena,64,AR,1584324185,95,-26.79,-60.44,71.330,2.16
561,borujerd,0,IR,1584324187,89,33.90,48.75,33.746,1.77
562,kenora,90,CA,1584324189,85,49.82,-94.43,24.800,2.60
563,marica,0,BR,1584324042,78,-22.92,-42.82,80.600,3.10


### Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Storing variables, adding a column to hotel_df
hotel_df = my_weather

#Save the config info
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="

index = 0

# Narrow down the DataFrame to find hotel locations within search parameters

hotels = [] #Defining hotels as list first

#Running loops to found the Hotel Names

for index, row in hotel_df.iterrows():
    query_url = url + str(row["Lat"]) + "," + str(row["Lng"]) + "&radius=5000&type=lodging&key=" + g_key
    response = requests.get(query_url).json()
    time.sleep(0.5)
    try:
        hotels.append(response["results"][0]["name"])
    except:
        hotels.append("Not Found")
        pass
        
hotel_df["Hotel Name"] = hotels


C:\Users\marib\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,port alfred,100,ZA,1584322882,98,-33.59,26.89,62.996,4.81,The Halyards Hotel
1,koulamoutou,52,GA,1584322884,97,-1.13,12.47,71.204,0.66,Motel Biki
2,tabuk,82,PH,1584322419,74,17.42,121.44,76.514,5.15,Grand Zion Garden Resort Hotel
3,nikolskoye,75,RU,1584322392,55,59.70,30.79,33.800,4.00,Tourist House - Sablino
4,fortuna,20,US,1584322458,70,40.60,-124.16,48.200,4.60,Super 8 By Wyndham Fortuna
...,...,...,...,...,...,...,...,...,...,...
560,presidencia roque saenz pena,64,AR,1584324185,95,-26.79,-60.44,71.330,2.16,Hotel Aconcagua
561,borujerd,0,IR,1584324187,89,33.90,48.75,33.746,1.77,Zagros International Hotel
562,kenora,90,CA,1584324189,85,49.82,-94.43,24.800,2.60,Maynard Lake Lodge
563,marica,0,BR,1584324042,78,-22.92,-42.82,80.600,3.10,Marica Hills


In [8]:
#Filtering the hotel DF

hotel_filtered_df = hotel_df.loc[hotel_df["Hotel Name"] != "Not Found", ["Hotel Name", "City", "Country", "Lat", "Lng", "Humidity"]]
hotel_filtered_df

,Hotel Name,City,Country,Lat,Lng,Humidity
0,The Halyards Hotel,port alfred,ZA,-33.59,26.89,98
1,Motel Biki,koulamoutou,GA,-1.13,12.47,97
2,Grand Zion Garden Resort Hotel,tabuk,PH,17.42,121.44,74
3,Tourist House - Sablino,nikolskoye,RU,59.70,30.79,55
4,Super 8 By Wyndham Fortuna,fortuna,US,40.60,-124.16,70
...,...,...,...,...,...,...
560,Hotel Aconcagua,presidencia roque saenz pena,AR,-26.79,-60.44,95
561,Zagros International Hotel,borujerd,IR,33.90,48.75,89
562,Maynard Lake Lodge,kenora,CA,49.82,-94.43,85
563,Marica Hills,marica,BR,-22.92,-42.82,78


In [9]:
# Plot markers on top of the heatmap

hotel_marker = []
hotel_dict_markers = {}
for index, row in hotel_filtered_df.iterrows():
    hotel_dict_markers["Hotel Name"] = row["Hotel Name"]
    hotel_dict_markers["location"] = (row["Lat"], row["Lng"])
    hotel_dict_markers["City"] = row["City"]
    hotel_dict_markers["Country"] = row["Country"]
    hotel_marker.append(hotel_dict_markers)
    hotel_dict_markers = {}

#Defining the info box template

cities = [city["location"] for city in hotel_marker]
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**city) for city in hotel_marker]

# Defining weights variable
humidity = hotel_filtered_df["Humidity"]

# Defining and adding hotel marker layer

marker_layer = gmaps.marker_layer(cities, info_box_content = hotel_info)
fig = gmaps.figure(center=(10.0, 10.0), zoom_level=1.6)
fig.add_layer(marker_layer)
fig.add_layer(gmaps.heatmap_layer(cities, weights = humidity, dissipating=False, 
                                  point_radius = 3.1, max_intensity = 150))

# Map layout and display
figure_layout = {'width': '400px','height': '600px','border': '1px solid black',
                 'padding': '1px','margin': '0 auto 0 auto'}
fig

Figure(layout=FigureLayout(height='420px'))